In [1]:
import numpy as np
from itertools import groupby


In [111]:
class_to_id = {'Pedestrian':0, 'Car':1, 'Van':2}

with open('data/0000.txt','r') as f:
    lines = f.readlines()
    all_labels = list()
    for line in lines:
        label_frame = line.split()
        all_labels.append(label_frame)
    grouped = group_by_first_element(np.array(all_labels))
    
    for key,group in grouped.items():
        frame_id = ('000000' + key )[-6:]
        for line in group:

            class_name = line[2]
            if class_name in class_to_id.keys():
                class_num = class_to_id[class_name]
                loc = np.array([[float(line[10]), float(line[11]), float(line[12])]])
                h = float(line[11])
                w = float(line[12])
                l = float(line[13])
                loc_lidar = calib.rect_to_lidar(loc)
                loc_lidar[:, 2] += h/2  
                
            
        #output format x,y,z,h,w,l,yaw,class
            

In [100]:
def group_by_first_element(arr):
    # 先对数组按每行的第一个元素进行排序
    arr_sorted = arr[arr[:, 0].argsort()]
    
    # 然后使用 groupby 进行分组
    grouped = {}
    for key, group in groupby(arr_sorted, lambda x: x[0]):
        grouped[key] = np.array(list(group))
    
    return grouped

# 示例输入
arr = np.array([
    [1, 2, 3],
    [2, 3, 4],
    [1, 4, 5],
    [3, 1, 1],
    [2, 2, 2]
])

# 调用分组函数
# grouped = group_by_first_element(arr)

# # 输出结果
# for key, group in grouped.items():
#     print(f"Group {key}:")
#     print(group)

In [ ]:
line.split()

In [ ]:
grouped = group_by_first_element(np.array(all_labels))

In [51]:
('12'+'23')[2]

'2'

In [53]:
'0000123'[-6:]

'000123'

In [57]:
a = np.array([])

In [151]:
-(np.pi/2 + loc[...,np.newaxis])

array([[[1498.42920367],
        [1498.42920367],
        [1495.92920367]]])

In [163]:
with open('data/test.txt','w') as fw:
    fw.writelines(a)

In [164]:
import numpy as np
from scipy.spatial import ConvexHull

def PolyArea2D(pts):
    """
    Calculate the area of a 2D polygon.
    :param pts: (N, 2) array of polygon vertices.
    :return: area of the polygon.
    """
    lines = np.hstack([pts, np.roll(pts, -1, axis=0)])
    return 0.5 * np.abs(np.sum(lines[:, 0] * lines[:, 3] - lines[:, 2] * lines[:, 1]))

def convex_area(boxa_bottom, boxb_bottom):
    """
    Calculate the convex area of the combined bottom corners of two bounding boxes.
    :param boxa_bottom: (4, 2) array of bottom corners of the first bounding box.
    :param boxb_bottom: (4, 2) array of bottom corners of the second bounding box.
    :return: convex area of the combined corners.
    """
    # Combine the bottom corners of both bounding boxes
    all_corners = np.vstack((boxa_bottom, boxb_bottom))
    
    # Compute the convex hull of the combined corners
    C = ConvexHull(all_corners)
    
    # Extract the vertices of the convex hull
    convex_corners = all_corners[C.vertices]
    
    # Calculate the area of the convex hull
    convex_area = PolyArea2D(convex_corners)
    
    return convex_area

# Example usage:
boxa_bottom = np.array([[0, 0], [1, 0], [1, 1], [0, 1]])
boxb_bottom = np.array([[2, 2], [3, 2], [3, 3], [2, 3]])
area = convex_area(boxa_bottom, boxb_bottom)
print(f"Convex area: {area}")


Convex area: 5.0


In [172]:
def calculate_iou(poly1, poly2):
    # 计算交集面积
    interArea = polygon_intersection_area(poly1, poly2)
    
    # 计算各自的面积
    poly1_area = polygon_area(poly1)
    poly2_area = polygon_area(poly2)

    # 计算IoU
    iou = interArea / float(poly1_area + poly2_area - interArea)

    return iou

def calculate_giou(poly1, poly2):
    # 计算IoU
    iou = calculate_iou(poly1, poly2)

    # 计算封闭框（convex hull）
    xC1 = min(np.min(poly1[:, 0]), np.min(poly2[:, 0]))
    yC1 = min(np.min(poly1[:, 1]), np.min(poly2[:, 1]))
    xC2 = max(np.max(poly1[:, 0]), np.max(poly2[:, 0]))
    yC2 = max(np.max(poly1[:, 1]), np.max(poly2[:, 1]))

    # 计算封闭框的面积
    convexArea = (xC2 - xC1) * (yC2 - yC1)

    # 计算GIoU
    giou = iou - (convexArea - (poly1_area + poly2_area - interArea)) / float(convexArea)

    return giou


In [181]:
def sutherland_hodgman_clip(subject_polygon, clip_polygon):
    """Clip a polygon with another polygon using the Sutherland-Hodgman algorithm."""
    def inside(p, edge_start, edge_end):
        return (edge_end[0] - edge_start[0]) * (p[1] - edge_start[1]) > (edge_end[1] - edge_start[1]) * (p[0] - edge_start[0])

    def compute_intersection(p1, p2, p3, p4):
        denom = (p1[0] - p2[0]) * (p3[1] - p4[1]) - (p1[1] - p2[1]) * (p3[0] - p4[0])
        if denom == 0:
            return None
        x = ((p1[0] * p2[1] - p1[1] * p2[0]) * (p3[0] - p4[0]) - (p1[0] - p2[0]) * (p3[0] * p4[1] - p3[1] * p4[0])) / denom
        y = ((p1[0] * p2[1] - p1[1] * p2[0]) * (p3[1] - p4[1]) - (p1[1] - p2[1]) * (p3[0] * p4[1] - p3[1] * p4[0])) / denom
        return np.array([x, y])

    output_list = subject_polygon
    for i in range(len(clip_polygon)):
        input_list = output_list
        output_list = []
        if len(input_list) == 0:
            break
        edge_start = clip_polygon[i]
        edge_end = clip_polygon[(i + 1) % len(clip_polygon)]
        for j in range(len(input_list)):
            current_point = input_list[j]
            prev_point = input_list[(j - 1) % len(input_list)]
            if inside(current_point, edge_start, edge_end):
                if not inside(prev_point, edge_start, edge_end):
                    output_list.append(compute_intersection(prev_point, current_point, edge_start, edge_end))
                output_list.append(current_point)
            elif inside(prev_point, edge_start, edge_end):
                output_list.append(compute_intersection(prev_point, current_point, edge_start, edge_end))
    return np.array(output_list)

In [2]:
import numpy as np

def polygon_area(vertices):
    """Calculate the area of a polygon given its vertices using the Shoelace formula."""
    x = vertices[:, 0]
    y = vertices[:, 1]
    return 0.5 * np.abs(np.dot(x, np.roll(y, 1)) - np.dot(y, np.roll(x, 1)))

def sutherland_hodgman_clip(subject_polygon, clip_polygon):
    """Clip a polygon with another polygon using the Sutherland-Hodgman algorithm."""
    def inside(p, edge_start, edge_end):
        return (edge_end[0] - edge_start[0]) * (p[1] - edge_start[1]) > (edge_end[1] - edge_start[1]) * (p[0] - edge_start[0])

    def compute_intersection(p1, p2, p3, p4):
        denom = (p1[0] - p2[0]) * (p3[1] - p4[1]) - (p1[1] - p2[1]) * (p3[0] - p4[0])
        if denom == 0:
            return None
        x = ((p1[0] * p2[1] - p1[1] * p2[0]) * (p3[0] - p4[0]) - (p1[0] - p2[0]) * (p3[0] * p4[1] - p3[1] * p4[0])) / denom
        y = ((p1[0] * p2[1] - p1[1] * p2[0]) * (p3[1] - p4[1]) - (p1[1] - p2[1]) * (p3[0] * p4[1] - p3[1] * p4[0])) / denom
        return np.array([x, y])

    output_list = subject_polygon
    for i in range(len(clip_polygon)):
        input_list = output_list
        output_list = []
        if len(input_list) == 0:
            break
        edge_start = clip_polygon[i]
        edge_end = clip_polygon[(i + 1) % len(clip_polygon)]
        for j in range(len(input_list)):
            current_point = input_list[j]
            prev_point = input_list[(j - 1) % len(input_list)]
            if inside(current_point, edge_start, edge_end):
                if not inside(prev_point, edge_start, edge_end):
                    intersection = compute_intersection(prev_point, current_point, edge_start, edge_end)
                    if intersection is not None:
                        output_list.append(intersection)
                output_list.append(current_point)
            elif inside(prev_point, edge_start, edge_end):
                intersection = compute_intersection(prev_point, current_point, edge_start, edge_end)
                if intersection is not None:
                    output_list.append(intersection)
    return np.array(output_list)

def calculate_iou(poly1_coords, poly2_coords):
    """Calculate the Intersection over Union (IOU) of two polygons."""
    poly1_coords = np.array(poly1_coords)
    poly2_coords = np.array(poly2_coords)

    # Calculate the intersection polygon
    intersection_polygon = sutherland_hodgman_clip(poly1_coords, poly2_coords)
    if len(intersection_polygon) < 3:  # Not a valid polygon if it has less than 3 vertices
        return 0.0

    # Calculate the area of each polygon
    area1 = polygon_area(poly1_coords)
    area2 = polygon_area(poly2_coords)
    intersection_area = polygon_area(intersection_polygon)

    # Calculate the union area
    union_area = area1 + area2 - intersection_area

    # Calculate IOU
    iou = intersection_area / union_area
    return iou

# 示例坐标
poly1_coords = [(0, 0), (2, 0), (2, 2), (0, 2)]
poly2_coords = [(1, 1), (3, 1), (3, 3), (1, 3)]

# 计算IOU
iou = calculate_iou(poly1_coords, poly2_coords)
print(f'IOU: {iou}')


IOU: 0.14285714285714285


In [32]:
poly1_coords = [(0, 0), (0, 2), (4, 2), (4, 0)]
poly2_coords = [(1, 0), (3, 0), (3, 4), (1, 4)]

# 计算IOU
iou = calculate_iou(poly1_coords, poly2_coords)
print(f'IOU: {iou}')

IOU: 0.3333333333333333


In [9]:
intersec = sutherland_hodgman_clip(poly1_coords, poly2_coords)
print(intersec)
area = polygon_area(intersec)
print(area)

[[ 1. -0.]
 [ 3.  0.]
 [ 3.  2.]
 [ 1.  2.]]
4.0


In [21]:
def is_clockwise(vertices):
    """Check if the polygon vertices are in clockwise order."""
    total = 0
    for i in range(len(vertices)):
        x1, y1 = vertices[i]
        x2, y2 = vertices[(i + 1) % len(vertices)]
        total += (x2 - x1) * (y2 + y1)
    return total > 0

def ensure_clockwise(vertices):
    """Ensure the vertices are in clockwise order."""
    if not is_clockwise(vertices):
        vertices = np.flipud(vertices)
    return vertices

In [35]:
polygon_area(np.array([[3,0],[1,0],[1,2],[3,2]]))

4.0

In [14]:
type(intersec[0])

numpy.ndarray

In [23]:
ensure_clockwise(np.array([[3,0],[3,2],[1,2],[1,-1]]))

array([[ 1, -1],
       [ 1,  2],
       [ 3,  2],
       [ 3,  0]])

In [33]:
sutherland_hodgman_clip(poly1_coords, poly2_coords)

array([[3., 0.],
       [1., 0.],
       [1., 2.],
       [3., 2.]])